## Dataset Preparation

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2


DATADIR = "E:\Download\pythonCourse\datasets\PetImages"
CATEGORIES = ['Dog', 'Cat']



In [7]:

#Reshape imgs
IMG_SIZE = 50

In [8]:
#Training dataset

training_data = []

def create_training_data():
  for category in CATEGORIES:
    path = os.path.join(DATADIR, category) # Path to cats or dogs dir
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path=path):
      #Create a full path with the img.
      #Then Convert the img to gray scale. 
      # REASONS:
      #RGB data is 3x the size of gray scale data.
      #Color is not relevant in this specific task.

      try:
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_array, class_num])
      except Exception as e:
        pass

create_training_data()

In [9]:
#In classification your training data should be properly
#balanced.
#In this case it should be 50-50 for cats and dogs 
#This means, just as many cats and just as many dogs
print(len(training_data))

22860


In [10]:
#it's important to shuffle the data
import random

#Shuffling the data
random.shuffle(training_data)

In [11]:
for sample in training_data[:10]:
  print(sample[1])


0
0
1
1
1
0
1
0
0
0


In [12]:
#Generally capital x is your future set.
#Lower case y is where your labels are.
X = []
y = []

In [13]:

for features, label in training_data:
  X.append(features)
  y.append(label)

#The features are converted to a list, but
#you can't pass a list to a neural network
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [14]:
#Chances are you will be just tweaking the model the next time
#So, you don't wanna to be rebuilding your dataset everytime
#That why you need to save your data afterwards

import pickle

#Saving x
pickle_out = open('X.pickle','wb')
pickle.dump(X, pickle_out)
pickle_out.close()


#Saving y
pickle_out = open('y.pickle','wb')
pickle.dump(y, pickle_out)
pickle_out.close()

In [15]:
#Later, if you want to read it back in:

pickle_in = open('X.pickle','rb')
X = pickle.load(pickle_in)

In [16]:
X[1]

array([[[148],
        [152],
        [153],
        ...,
        [152],
        [152],
        [151]],

       [[153],
        [156],
        [156],
        ...,
        [160],
        [167],
        [153]],

       [[155],
        [157],
        [160],
        ...,
        [160],
        [166],
        [160]],

       ...,

       [[216],
        [217],
        [220],
        ...,
        [108],
        [141],
        [120]],

       [[220],
        [223],
        [225],
        ...,
        [109],
        [130],
        [180]],

       [[223],
        [222],
        [225],
        ...,
        [102],
        [186],
        [195]]], dtype=uint8)

## Convolutional Neural Networkd - Using Dataset

 - [Part3](https://www.youtube.com/watch?v=WvoLTXIjBYU&list=PLQVvvaa0QuDfhTox0AjmQ6tvTgMBZBEXN&index=4)

In [26]:
import tensorflow as tf 
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
# from keras.layers.convolutional import Conv2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle
import time

NAME = 'Cats-vs-dog-cnn-62x2-{}'.format(int(time.time()))
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
# sess = tf.Sessio

X = pickle.load(open('X.pickle','rb'))
y = pickle.load(open('y.pickle','rb'))
y = np.array(y)
#Consider normalizing the data
#In imagery data, it varies from 0 to 255, so we divide it by 255

X = X/255.0

#Building the a Sequetial model 

model = Sequential()
#We're skiping the first column
model.add(Conv2D(64, (3,3), input_shape=X.shape[1:]) )
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)) )
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
  )

model.fit(X, y, batch_size=32, epochs=4, validation_split=0.1)

Epoch 1/4
643/643 [==============================] - 114s 176ms/step - loss: 0.6007 - accuracy: 0.6713 - val_loss: 0.5420 - val_accuracy: 0.7323
Epoch 2/4
643/643 [==============================] - 91s 142ms/step - loss: 0.5080 - accuracy: 0.7551 - val_loss: 0.4769 - val_accuracy: 0.7664
Epoch 3/4
643/643 [==============================] - 93s 145ms/step - loss: 0.4578 - accuracy: 0.7847 - val_loss: 0.4622 - val_accuracy: 0.7743
Epoch 4/4
643/643 [==============================] - 93s 145ms/step - loss: 0.4113 - accuracy: 0.8128 - val_loss: 0.4555 - val_accuracy: 0.7927


## Analysing Models

[Tensorboard](https://www.youtube.com/watch?v=BqgTU7_cBnk&list=PLQVvvaa0QuDfhTox0AjmQ6tvTgMBZBEXN&index=4)

In [28]:
import tensorflow as tf 
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
# from keras.layers.convolutional import Conv2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

NAME = 'Cats-vs-dog-cnn-62x2-{}'.format(int(time.time()))

tensorboard = TensorBoard(log_dir=f'logs/{NAME}')

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
# sess = tf.Sessio

X = pickle.load(open('X.pickle','rb'))
y = pickle.load(open('y.pickle','rb'))
y = np.array(y)
#Consider normalizing the data
#In imagery data, it varies from 0 to 255, so we divide it by 255

X = X/255.0

#Building the a Sequetial model 

model = Sequential()
#We're skiping the first column
model.add(Conv2D(64, (3,3), input_shape=X.shape[1:]) )
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)) )
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
  )

model.fit(X, y, batch_size=32, epochs=10, 
validation_split=0.1, callbacks=[tensorboard])


Epoch 1/10
643/643 [==============================] - 143s 220ms/step - loss: 0.6212 - accuracy: 0.6481 - val_loss: 0.5327 - val_accuracy: 0.7323
Epoch 2/10
643/643 [==============================] - 95s 148ms/step - loss: 0.5149 - accuracy: 0.7464 - val_loss: 0.4873 - val_accuracy: 0.7520
Epoch 3/10
643/643 [==============================] - 95s 147ms/step - loss: 0.4648 - accuracy: 0.7778 - val_loss: 0.4727 - val_accuracy: 0.7690
Epoch 4/10
643/643 [==============================] - 95s 148ms/step - loss: 0.4297 - accuracy: 0.7995 - val_loss: 0.4617 - val_accuracy: 0.7830
Epoch 5/10
643/643 [==============================] - 96s 149ms/step - loss: 0.3911 - accuracy: 0.8214 - val_loss: 0.4441 - val_accuracy: 0.7861
Epoch 6/10
643/643 [==============================] - 96s 149ms/step - loss: 0.3476 - accuracy: 0.8460 - val_loss: 0.5038 - val_accuracy: 0.7690
Epoch 7/10
643/643 [==============================] - 97s 150ms/step - loss: 0.2994 - accuracy: 0.8703 - val_loss: 0.4784 - val_a

## Optimizing With TensorBoard


In [30]:


import tensorflow as tf 
# from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
# from keras.layers.convolutional import Conv2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time


tensorboard = TensorBoard(log_dir=f'logs/{NAME}')

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
# sess = tf.Sessio

X = pickle.load(open('X.pickle','rb'))
y = pickle.load(open('y.pickle','rb'))
y = np.array(y)
#Consider normalizing the data
#In imagery data, it varies from 0 to 255, so we divide it by 255

X = X/255.0

dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
  for layer_size in layer_sizes:
    for conv_layer in conv_layers:
      NAME = f'{conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense-{int(time.time())}'
      #Building the a Sequential model 
      model = Sequential()
      #We're skiping the first column
      model.add(Conv2D(layer_size, (3,3), input_shape=X.shape[1:]) )
      model.add(Activation('relu'))
      model.add(MaxPooling2D(pool_size=(2,2)))

      for l in range(conv_layer-1):
        model.add(Conv2D(layer_size, (3,3)) )
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))

      model.add(Flatten())
      for l in range(dense_layer):
        model.add(Dense(dense_layer))
        model.add(Activation('relu'))

      model.add(Dense(1))
      model.add(Activation('sigmoid'))

      model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
        )

      model.fit(X, y, batch_size=32, epochs=10, 
      validation_split=0.1, callbacks=[tensorboard])


1-conv-32-nodes-0-dense-1633468739
2-conv-32-nodes-0-dense-1633468739
3-conv-32-nodes-0-dense-1633468739
1-conv-64-nodes-0-dense-1633468739
2-conv-64-nodes-0-dense-1633468739
3-conv-64-nodes-0-dense-1633468739
1-conv-128-nodes-0-dense-1633468739
2-conv-128-nodes-0-dense-1633468739
3-conv-128-nodes-0-dense-1633468739
1-conv-32-nodes-1-dense-1633468739
2-conv-32-nodes-1-dense-1633468739
3-conv-32-nodes-1-dense-1633468739
1-conv-64-nodes-1-dense-1633468739
2-conv-64-nodes-1-dense-1633468739
3-conv-64-nodes-1-dense-1633468739
1-conv-128-nodes-1-dense-1633468739
2-conv-128-nodes-1-dense-1633468739
3-conv-128-nodes-1-dense-1633468739
1-conv-32-nodes-2-dense-1633468739
2-conv-32-nodes-2-dense-1633468739
3-conv-32-nodes-2-dense-1633468739
1-conv-64-nodes-2-dense-1633468739
2-conv-64-nodes-2-dense-1633468739
3-conv-64-nodes-2-dense-1633468739
1-conv-128-nodes-2-dense-1633468739
2-conv-128-nodes-2-dense-1633468739
3-conv-128-nodes-2-dense-1633468739
